# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096944


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:40,  1.40s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:32,  1.21s/it]

Rendering models:  19%|█▉        | 6/31 [00:04<00:21,  1.16it/s]

Rendering models:  26%|██▌       | 8/31 [00:07<00:23,  1.02s/it]

...

Rendering models:  77%|███████▋  | 24/31 [00:08<00:01,  6.71it/s]

Rendering models:  84%|████████▍ | 26/31 [00:08<00:00,  7.72it/s]

Rendering models:  94%|█████████▎| 29/31 [00:08<00:00,  9.87it/s]

Rendering models: 100%|██████████| 31/31 [00:08<00:00, 10.52it/s]

not-logged-in-3b0b929f4475fd276561    0.001594
Jnursssmith5263                       0.000861
HM01                                  0.000320
jnarayanbvg                           0.000260
awright5                              0.000905
ElisabethB                            0.011726
AaronWallage117                       0.010782
jaksonA                               0.417793
framosuw                              0.000221
not-logged-in-9454a205d149d7907cd0    1.611661
dxjdxjhhhh                            0.000444
iesfranciscoayala                     0.000655
gabrielweiss                          0.000313
databanana                            0.000146
not-logged-in-3ae935e3afaee843f530    0.001963
Lavadude                              0.031266
not-logged-in-f0338f45b1f60a64aa34    0.000848
TanlerP                               0.011130
hunchoquavo                           3.177854
not-logged-in-d609ce6cbb13c8f911bd    0.005828
rmacmi2                               0.000188
syt18        

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())